In [6]:
import pandas as pd
import glob
pd.set_option("display.max_rows", 30)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_colwidth", 1000)
pd.set_option("display.width", 2000)

In [7]:
import gensim

In [8]:
import numpy as np
np.set_printoptions(precision=4,suppress=True)
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
import spacy

In [10]:
frames = []
for f in glob.glob('../data/*.csv'):
    frames.append(pd.read_csv(f))

In [11]:
raw_df = pd.concat(frames)

In [12]:
raw_df.query('(kind==1 or kind==3) and SichterName').Content.head(5)

2     Muss ich mit Hotspot registrieren wlan to go a...
14    Hallo zusammen . Gab es vor kurzem ein Update ...
22    Seit gestern geht wieder einmal nichts ( Kein ...
30    Könnten Sie sich bitte einen anderen Kooperati...
34    was die telekom mit ihr kunden abzieht ist unt...
Name: Content, dtype: object

In [13]:
raw_df

,SichterGruppe,SichterName,Host,kind,Datetime,Content,Url,Authorid,Tag,entryid,Id,TwistId,TopicId,TopicName,IsInitialEntry,IsPrivate
0,NaN,NaN,twitter.com,1,21.10.2016 23:18:00,HighTechno ! Ich will Claras WG mal so richtig...,https://twitter.com/Carina42008/status/7895764...,3796107.0,NaN,bc6c4c70-a00d-429d-8bde-4e7afa072a15,31796013,04b9f324-5cee-4dd3-8387-0ff934ec1472,551631,Alle,False,False
1,NaN,NaN,twitter.com,3,21.10.2016 23:18:00,"Yeah , der WG gefällt deine Musik . Hol dir au...",https://twitter.com/telekomerleben/status/7895...,1060906.0,NaN,5a676fbd-f7c7-45b1-8744-3a7fd8ebc9e9,31795989,04b9f324-5cee-4dd3-8387-0ff934ec1472,551631,Alle,True,False
2,Sichter FB/TW,Hotspot,www.facebook.com,1,24.09.2016 14:57:02,Muss ich mit Hotspot registrieren wlan to go a...,https://www.facebook.com/122768271103611/posts...,7707299.0,NaN,94e8d4f1-60c1-45e5-894a-c11a3ff7b655,30744441,0be928df-7378-421e-be6e-07269f1dac19,551631,Alle,True,False
3,NaN,NaN,www.facebook.com,3,24.09.2016 15:12:35,Hallo Fiona . Wenn Sie für WLAN TO GO angemeld...,https://www.facebook.com/122768271103611/posts...,254017.0,NaN,263da5f3-4f0c-4a3e-ac27-358eb033912f,30744872,0be928df-7378-421e-be6e-07269f1dac19,551631,Alle,False,False
4,NaN,NaN,www.facebook.com,4,24.09.2016 15:16:45,"ich bin nicht gemeldet , es ok ich vergesse es",https://www.facebook.com/122768271103611/posts...,7707299.0,NaN,51a8de05-48e4-4422-822f-50aba72fb5f5,30745002,0be928df-7378-421e-be6e-07269f1dac19,551631,Alle,False,False
5,Sichter FB/TW,Hotspot,www.facebook.com,4,24.09.2016 15:18:01,Jetzt muss ich mal eins fragen . Ich kann doch...,https://www.facebook.com/122768271103611/posts...,127469.0,NaN,a255ed96-818f-42b2-84f1-0c0c7e2581da,30745045,0be928df-7378-421e-be6e-07269f1dac19,551631,Alle,False,False
6,NaN,NaN,www.facebook.com,4,24.09.2016 15:18:01,Jetzt muss ich mal eins fragen . Ich kann doch...,https://www.facebook.com/122768271103611/posts...,127469.0,NaN,a255ed96-818f-42b2-84f1-0c0c7e2581da,30745045,0be928df-7378-421e-be6e-07269f1dac19,551631,Alle,False,False
7,NaN,NaN,www.facebook.com,4,24.09.2016 15:19:10,ich weis nicht wie es geht :(,https://www.facebook.com/122768271103611/posts...,7707299.0,NaN,53632505-a61d-423e-bfc2-9f2047ad75b1,30745129,0be928df-7378-421e-be6e-07269f1dac19,551631,Alle,False,False
8,NaN,NaN,www.facebook.com,3,24.09.2016 15:28:31,Natürlich dürfen Sie alle HotSpots der Telekom...,https://www.facebook.com/122768271103611/posts...,254017.0,NaN,5c21050e-f459-48c8-bd1d-d80b8b9888a3,30745333,0be928df-7378-421e-be6e-07269f1dac19,551631,Alle,False,False
9,NaN,NaN,www.facebook.com,4,24.09.2016 15:30:05,Na erst wenn das Vectoring hier abgeschlossen ...,https://www.facebook.com/122768271103611/posts...,127469.0,NaN,5591da8f-8679-4a9d-874d-6c5d3ad8b171,30745484,0be928df-7378-421e-be6e-07269f1dac19,551631,Alle,False,False


In [14]:
pd.unique(raw_df.SichterName)

array([nan, 'Hotspot', 'Festnetz', 'Mobilfunk', 'GK', 'MMS', 'A-Team',
       'Mobile', 'MMS COM', 'Mobilfunk (Mark./T-Shop)', 'Mobilfunk COM',
       'Festnetz (Mark./T-Shop)', 'Festnetz COM', 'MMS (Mark./T-Shop)',
       'Hybrid FB/TW', 'Wechsler', 'Smart Home FB/TW', 'Meinungsbildner',
       'Hosting FB/TW', 'Hybrid & LTE', 'Abuse/Missbrauch',
       'Anfragen in Arabisch', 'Entertain', 'IP & FTTH & PSTN',
       'Neue Themen', 'Dienste & Abos', 'Hosting', 'Endgeräte'],
      dtype=object)

In [15]:
from collections import defaultdict

def f(x):
     return pd.Series(dict(SichterName = set(x.SichterName) - {np.nan},
                                   Tag = set(x.Tag),
                               Content = list(x.Content),
                                  Kind = list(x.kind)))

df = raw_df.query('(kind==1 or kind==3) and SichterName').sort_values(['Datetime'],ascending=True).groupby('TwistId').apply(f)

In [16]:
df

,Content,Kind,SichterName,Tag
TwistId,,,,
000010ca-a611-423d-a6a7-375521e53431,[@Telekom_hilft Ich hab da mal ein Eltern-Weih...,[1],{Festnetz},{nan}
00006e01-e8ef-4677-8ca3-457ea8a4e3ac,[SO GEHT MAN NICHT MIT SEINEN KUNDEN UM ! ! ! ...,[1],{Mobilfunk},{Leerkategorie}
0001d985-62d5-41bd-9e2c-c8a5af928207,"[Sehr geehrte Telekom , Durch den sehr schlech...",[1],{Mobilfunk},{nan}
00026ddd-f481-4935-9cb6-0fafac67bc38,[RT : Für ICE 27 ist noch zuständig . / da],[1],{Hotspot},{nan}
0004f38f-a51a-4a33-b1f7-754437e5e514,[Hallo . Sind aktuell Störungen im Raum 012345...,[1],{MMS},{nan}
00055016-2103-4240-8979-ec9a0048e4c9,"[Darauf warten , dass der T-Punkt aufmacht ......",[1],{Mobilfunk},{nan}
00087ad4-ffb7-42c7-947e-efcd7531cc29,"[Hallo @Telekom_hilft - "" EntertainTV mobil "" ...",[1],{Festnetz},{nan}
00090c82-7bd0-48e7-b688-28d148259a06,"[... so liebe Telekom , nach 12 Tagen wurde me...",[1],{GK},{nan}
00097026-31f3-4a10-a8d2-f681f5a94cf4,"[@Telekom_hilft Bin in England , versuche eing...",[1],{MMS},{nan}


In [331]:
import re
GRUBER_URLINTEXT_PAT = re.compile(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?\xab\xbb\u201c\u201d\u2018\u2019]))')
NUMBER_CLEAN = re.compile(r'\d+(:?[.,]?\d+)*')
TIME_CLEAN = re.compile(r'\d+:\d+(?:\s?[u]hr)?')
PUNCT_CLEAN = re.compile(r'(\s?)[\.\,()\[\]\{\}\^\:\'\"\/;/!/?/…/€/%//“=–><&]+(\s?)')
DASH_CLEAN = re.compile(r'([^et])[-]')
TWEET_CLEAN = re.compile(r'@\w+')
HASH_CLEAN = re.compile(r'#(\w+)')
SPACE_CLEAN = re.compile(r'(\s)\s+')
RT_CLEAN = re.compile(r'^rt\b')

def preprocessing(text):
    try:
        text = text.lower()
        # clean-up url
        text = GRUBER_URLINTEXT_PAT.sub('', text)
        # remove time
        text = TIME_CLEAN.sub('', text)
        # clean-up numbers
        text = NUMBER_CLEAN.sub('', text)
        text = PUNCT_CLEAN.sub(r'\1\2',text)
        text = TWEET_CLEAN.sub('', text)
        text = HASH_CLEAN.sub(r'\1', text)
#         text = DASH_CLEAN.sub(r'\1 ', text)
        text = SPACE_CLEAN.sub(r'\1', text)
        text = RT_CLEAN.sub(r'', text)
        return ' '.join([word for word in text.split(' ') if word not in stopwords]).strip()
    except:
        return ''

with open('./stopwords-de.txt') as f:
    stopwords = f.read().split('\n')

In [335]:
documents = [] # Content
documents_raw = []
labels = [] # SichterName

for (idx, row) in df.iterrows():
#     if not row.Content:
#         continue
    labels.append(row.SichterName)
    documents_raw.append(row.Content)
    documents.append(' '.join([preprocessing(text) for text in documents_raw[-1] if text]))
#     docs.append([nlp(document) for document in documents[-1]])

print(len(documents))

35565


In [116]:
documents[:30]

['eltern weihnachten technik problem',
 'kunden methoden anf juni handyvertrag iphone abgeschlossen werbung sohnes gutschrift ablauf rückgaberechts zugesagt sohn werbungsprämie rechnung kamen gutschrift verrechnet schrieb mail zuständigen sachbearbeiter name email adresse mail reaktion lt sachbearbeiter mails spamordner gelandet rief zuständige sachbearbeiter zurückrief prämien erneut zugesagt gutschrift abrechnung gutschriften gutschriften zugesagt vertrag nieee stande gekommen gestern ca minuten warteschleife beschwerdehotline gesprochen sofortige bestätigung zumindest gutschrift zugesagt sofort per mail bestätigen methoden kunden versprechungen einhalten',
 'geehrte gewordenen kunden service leider vertrag kündigen gehalten versprochen zudem wechseln tarifpartner',
 'ice zuständig',
 'aktuell störungen raum festnetztelefonie funktioniert stunden router mehrmals neu gestartet gruß jens',
 'warten t-punkt aufmacht kpauttesiphone hilflos lasttweet',
 'entertaintv mobil magenta kostenlo

In [117]:
from gensim import corpora, models

dictionary = corpora.Dictionary([document.split(' ') for document in documents])

In [118]:
corpus = [dictionary.doc2bow(document.split(' ')) for document in documents]

In [119]:
N_topics = 30
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=N_topics, id2word=dictionary, passes=20)

In [155]:
a = ldamodel[dictionary.doc2bow(documents[10].split(' '))]

In [151]:
documents[10]

'hallöchen frage sieht thema entschädigung kunden woch internet telefon mitarbeiter anschluss fit logischerweise woche grundleistung weggefallen möglichkeiten entgegen seitens'

In [161]:
sorted(a, key=lambda x: x[1],reverse=True)[0][0]

[(15, 0.6417565317031665),
 (19, 0.15547725295686213),
 (2, 0.10417424262521385),
 (10, 0.05525863938139948)]

In [154]:
print(ldamodel.print_topics(num_topics=30, num_words=1))

[(0, '0.038*"&"'), (1, '0.037*"iphone"'), (2, '0.042*"erreichbar"'), (3, '0.153*"dsl"'), (4, '0.035*"kundencenter"'), (5, '0.012*"hotspots"'), (6, '0.047*"eigentlich"'), (7, '0.040*"techniker"'), (8, '0.028*"😂"'), (9, '0.060*"störung"'), (10, '0.026*"problem"'), (11, '0.039*"the"'), (12, '0.079*"netz"'), (13, '0.084*"entertain"'), (14, '0.018*"kunden"'), (15, '0.034*"rechnung"'), (16, '0.032*"adresse"'), (17, '0.078*"vertrag"'), (18, '0.057*"*"'), (19, '0.120*"internet"'), (20, '0.054*"vdsl"'), (21, '0.038*"störungen"'), (22, '0.097*"wlan"'), (23, '0.027*"via"'), (24, '0.074*"magenta"'), (25, '0.040*"😡"'), (26, '0.034*"leute"'), (27, '0.043*"leitung"'), (28, '0.084*"router"'), (29, '0.024*"per"')]


In [163]:
# bag of words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

BoW = CountVectorizer(documents+qa, stop_words=stopwords, strip_accents='unicode', ngram_range=(1,3), min_df=3)
X_bag = BoW.fit_transform(documents+qa)
tf_transformer = TfidfTransformer()
X_bag = tf_transformer.fit_transform(X_bag)

In [216]:
vectors = []
clusters = []
for document in documents:
    cluster = ldamodel[dictionary.doc2bow(document.split(' '))]
    clusters.append(sorted(a, key=lambda x: x[1],reverse=True)[0][0])
    vector = BoW.transform([document])
    vector = tf_transformer.transform(vector)
    vectors.append(vector)
    
qa_vectors = []
for document in qa:
    vector = BoW.transform([document])
    vector = tf_transformer.transform(vector)
    qa_vectors.append(vector)

In [217]:
from sklearn.metrics.pairwise import cosine_similarity

In [218]:
from scipy.sparse import vstack

In [219]:
vectors = vstack(vectors)

In [220]:
qa_vectors = vstack(qa_vectors)

In [223]:
vectors.shape

(35565, 57865)

In [271]:
labels[7135]

{'Festnetz'}

In [273]:
qa_labels[-3]

['geraete-zubehoer',
 'telefone-und-anlagen',
 'speedphone',
 'anmeldung-des-speedphone-am-speedport-router']

In [289]:
def get_topic(document):
    topics = ldamodel[dictionary.doc2bow(document.split(' '))]
    topic_ind = sorted(topics, key=lambda x: x[1],reverse=True)[0][0]
    return topic_ind, ldamodel.print_topics(num_topics=30, num_words=10)[topic_ind]

In [243]:
from sklearn.metrics.pairwise import linear_kernel
sim = linear_kernel(vectors, qa_vectors)
sim.shape

(35565, 1655)

In [299]:
sort_sim = sim.argsort(axis=0)
sort_sim.shape

(35565, 1655)

In [304]:
sort_sim[-1,:].shape

(1655,)

In [ ]:
for cluster in range(N_topics):
    topics==cluster

In [ ]:
def get_qa(document_ind):
    sort_sim[-1:document_ind]

In [285]:
get_topic(documents[7135])

(23, '0.027*"via" + 0.015*"inkl" + 0.014*"ergebnis" + 0.013*"bandbreite" + 0.011*"homepage" + 0.009*"betroffen" + 0.009*"minute" + 0.009*"nichtmal" + 0.008*"gen" + 0.008*"führt"')


(23, None)

In [302]:
topics = [get_topic(document)[0] for document in documents]

In [336]:
from collections import defaultdict
new_labels = defaultdict(list)
new_topics = defaultdict(list)

for i, ind in enumerate(sort_sim[-1,:]):
    new_labels[ind].append(qa_labels[i])
    new_topics[get_topic(documents[ind])].append(i)

In [328]:
documents[283]

'schöne neuerungen wi fi calling warte'

In [329]:
new_labels

defaultdict(list,
            {8: [['festnetz-internet-tv',
               'ip-basierter-anschluss',
               'leistungsmerkmale-mit-telefontasten-steuern']],
             37: [['auftrag-erste-schritte',
               'wechsel-zur-telekom',
               'techniker-vor-ort-notwendig'],
              ['festnetz-internet-tv',
               'entertain-tv',
               'entertaintv-neue-geraete',
               'entertaintv-installation-schritt-fuer-schritt'],
              ['festnetz-internet-tv',
               'anschluss-verfuegbarkeit',
               'anschlussvarianten',
               'allgemein',
               'aktuelle-dsl-geschwindigkeit-messen'],
              ['festnetz-internet-tv',
               'entertain-tv',
               'entertaintv-neue-geraete',
               'entertaintv-installation-schritt-fuer-schritt'],
              ['geraete-zubehoer', 'router', 'konfiguration-speedport'],
              ['geraete-zubehoer', 'router', 'konfiguration-speedport'],
 

In [318]:
len(qa_labels)

1655

In [319]:
len(topics)

35565

In [295]:
documents[7135], qa_labels[1403], qa[1403]

IndexError: list index out of range

In [122]:
import pickle
items = pickle.load(open('./cleaned_answers_2.pickle','rb'))

In [317]:
from itertools import product

qa = []
qa_labels = []

for item in items:
    KQ = [kq.decode('utf-8') for kq in item['key_questions']]
    answer = item['answer'].decode('utf-8')
    question = item['question'].decode('utf-8')
    if KQ:
        combs = product([question], KQ, [answer])
    else:
        combs = product([question], [answer])
    qa_labels += [item['link'].split('>')] * len(list(combs))
    qa += [' '.join(comb) for comb in combs]

In [ ]:
from collections import Counter
# create lemma lists filter punctuation and numbers
# from spacy
lemmas = [word.lemma_ for ldocs in docs for words in ldocs for word in words if word.pos_ not in ['PUNCT','NUM']]
counts = Counter(lemmas)

In [ ]:
lemma_df = pd.DataFrame.from_dict(counts, orient='index')
lemma_df.sort_values(by=0,ascending=False).head(50).plot(kind='bar', figsize=(20,10))

# Feature Analysis

In [ ]:
lemmas = [sum([len(word) for word in words]) for words in documents]
word_counts = Counter(lemmas)

lemma_df = pd.DataFrame.from_dict(word_counts, orient='index')
a = lemma_df.sort_values(by=0,ascending=False).head(50).plot(kind='bar', figsize=(20,10))
a.set_title('number of words per row')
a.set_ylabel('number of rows')
a.set_xlabel('number of words')

In [ ]:
lemmas = [sum([len(word) for word in words]) for words in documents]
word_counts = Counter(lemmas)

lemma_df = pd.DataFrame.from_dict(word_counts, orient='index')
a = lemma_df.sort_values(by=0,ascending=False).head(50).plot(kind='bar', figsize=(20,10))
a.set_title('number of characters per row')
a.set_ylabel('number of rows')
a.set_xlabel('number of words')

In [ ]:
from wordcloud import WordCloud
cloud = WordCloud(width=1440, height=1080).generate(" ".join([word.lemma_ for doc in docs for words in doc for word in words]))
plt.figure(figsize=(20, 15))
plt.imshow(cloud)
plt.title('Most common lemmas')
plt.axis('off')

In [ ]:
with open('./stopwords-de.txt') as f:
    stopwords = f.read().split('\n')

In [ ]:
from wordcloud import WordCloud
cloud = WordCloud(width=1440, height=1080).generate(" ".join([word.lemma_ for doc in docs for words in doc for word in words if word.lemma_ not in stopwords]))
plt.figure(figsize=(20, 15))
plt.imshow(cloud)
plt.title('Most common lemmas without stopwords')
plt.axis('off')

In [ ]:
qmarks = np.mean(df.Content.apply(lambda x: max(['?' in y for y in x])))
math = np.mean(df.Content.apply(lambda x: max(['€' in y for y in x])))
fullstop = np.mean(df.Content.apply(lambda x:  max(['.' in y for y in x])))
capital_first = np.mean(df.Content.apply(lambda x: x[0][0].isupper()))
capitals = np.mean(df.Content.apply(lambda x: max([tok.isupper() for y in x for tok in y.split(' ')])))
numbers = np.mean(df.Content.apply(lambda x: max([char.isdigit() for y in x for char in y])))

print('Rows with question marks: {:.2f}%'.format(qmarks * 100))
print('Rows with €: {:.2f}%'.format(math * 100))
print('Rows with full stops: {:.2f}%'.format(fullstop * 100))
print('Rows with capitalised first letters: {:.2f}%'.format(capital_first * 100))
print('Rows with capital letters: {:.2f}%'.format(capitals * 100))
print('Rows with numbers: {:.2f}%'.format(numbers * 100))

In [ ]:
# TODO: try different sets of stopwords (e.g. with or without kein, nicht)

In [ ]:
# wordvectors:
w2v = [[word.vector for word in doc if word.text not in stopwords] for doc in docs]

# average w2v
X = np.array([np.mean([word.vector for word in doc if word.text not in stopwords], 0) for doc in docs])

In [ ]:
# bag of words
from sklearn.feature_extraction.text import CountVectorizer

CountVectorizer()
BoW = CountVectorizer(documents)

In [ ]:
X_bag = BoW.fit_transform(documents)

In [ ]:
X_bag[0]

In [ ]:
Y = np.array(prios,dtype=int)

In [ ]:
spacy.info('de')

In [ ]:
import xgboost as xgb

# Set our parameters for xgboost
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.02
params['max_depth'] = 4

In [ ]:
# Code for cross-validation
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.utils import class_weight
from sklearn.metrics import f1_score

# Create 2 folds
k = 2
kfold = StratifiedKFold(Y, k, shuffle=True, random_state=0)

results = np.zeros(k)

# iterate over two folds
for i, (train_ind, test_ind) in enumerate(kfold):
    X_train, X_val, Y_train, Y_val = X_bag[train_ind], X_bag[test_ind], Y[train_ind], Y[test_ind]
    gnb = GaussianNB()
    D_train = xgb.DMatrix(X_train, label=Y_train)
    D_val = xgb.DMatrix(X_val, label=Y_val)

    weights = class_weight.compute_sample_weight('balanced', [0,1], Y_train)
    weightss = []
    for y in Y_train:
        weightss.append(weights[y])

#     predictions = gnb.fit(X_train, Y_train, sample_weight=weightss).predict(X_val)
    watchlist = [(D_train, 'train'), (D_val, 'valid')]

    bst = xgb.train(params, D_train, 400, watchlist, early_stopping_rounds=50, verbose_eval=10)
    predictions = bst.predict(data=D_val).round()
    results[i] = f1_score(predictions, Y_val)
    print('f1 score: {}'.format(results[i]))
    print('accuracy: {}\%'.format(sum(predictions==Y_val)/len(Y_val)*100))

In [ ]:
X_bag.shape

In [ ]:
from scipy.cluster.hierarchy import ward, dendrogram
from sklearn.metrics.pairwise import cosine_similarity

dist = 1 - cosine_similarity(X)
linkage_matrix = ward(dist) #define the linkage_matrix using ward clustering pre-computed distances

fig, ax = plt.subplots(figsize=(15, 20)) # set size
ax = dendrogram(linkage_matrix, orientation="right", labels=np.arange(X.shape[0]));

plt.tick_params(\
    axis= 'x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off')

plt.tight_layout() #show plot with tight layout

#uncomment below to save figure
plt.savefig('ward_clusters.png', dpi=200) #save figure as ward_clusters

In [ ]:
np.array(documents)[[17,9]]

In [ ]:
dist = 1 - cosine_similarity(X_bag)
linkage_matrix = ward(dist) #define the linkage_matrix using ward clustering pre-computed distances

fig, ax = plt.subplots(figsize=(15, 20)) # set size
ax = dendrogram(linkage_matrix, orientation="right", labels=np.arange(X.shape[0]));

plt.tick_params(\
    axis= 'x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off')

plt.tight_layout() #show plot with tight layout

#uncomment below to save figure
plt.savefig('ward_clusters.png', dpi=200) #save figure as ward_clusters

In [ ]:
linkage_matrix

In [ ]:
ward(dist)